# Recommender by ALS

With Collaborative filtering, we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B’s opinion on a different issue x than to have the opinion on x of a user-chosen randomly.

At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user’s rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user.

Matrix factorization: is a class of collaborative filtering algorithms used in recommender systems. Matrix factorization algorithms work by decomposing the user-item interaction matrix into the product of two lower dimensionality rectangular matrices.

Alternating least square(ALS) matrix factorization: The idea is basically to take a large (or potentially huge) matrix and factor it into some smaller representation of the original matrix through alternating least squares. We end up with two or more lower dimensional matrices whose product equals the original one.ALS comes inbuilt in Apache Spark.


In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
from os import path

ROOT_DIR = "./"
DATA_DIR = path.join(ROOT_DIR, 'data')
MODEL_DIR = path.join(ROOT_DIR, 'model')

In [11]:
"""
Utilities definition
"""

import hashlib

class Utils:
    
    '''
    TODO Transform the ID to unique integer
    '''
    @staticmethod
    def hashToInt(s):
        return int(hashlib.sha1(s).hexdigest(), 16) % (10 ** 8)

#Test
Utils.hashToInt('A141HP4LYPWMSR')

5460385L

## 1. Data preprocessing

In [4]:
""" 
Read ratings data from file to RDD
Split each line into 4 parts separated by the commas 
"""

path = os.path.join(DATA_DIR, 'ratings')
data = sc.textFile(path).map(lambda l: l.split(','))

In [5]:
"""
Create rdd named 'ratings' which is:
    ` A set of 'Ratings' object
    ` Combination of 3 features: product id as Int, user id as Int, rating score as float 
"""

from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

ratings = data.map(lambda (i, p, u, r): Rating(Utils.hashToInt(u), Utils.hashToInt(p), float(r)))

In [7]:
"""

Store userId in Int and movieIn in Int data 
    corresponding to their origin for later usage
    
"""

users = data.map(lambda (i, p,u,r): (Utils.hashToInt(u), u)).distinct().toDF()
movies = data.map(lambda (i, p, u, r): (Utils.hashToInt(p), p)).distinct().toDF()

user_path = os.path.join(DATA_DIR, 'users')
movie_path = os.path.join(DATA_DIR,'movies')

users.write.csv(user_path, mode='overwrite', compression='gzip')
movies.write.csv(movie_path, mode='overwrite', compression='gzip')


In [12]:
"""

Split dataset to trainset, testset
    ` Trainset: 80% random records
    ` Testset: 20% remained
    
X_train: movie and user of each review from Trainset
X_test: movie and user of each review from Testset

"""

(train_data, test_data) = ratings.randomSplit([0.8, 0.2])

X_train = train_data.map(lambda (p, u, r): (int(p), int(u)))

X_test = test_data.map(lambda (p, u, r): (int(p), int(u)))


In [27]:
train_data.count(), train_data.take(10)

(6330667,
 [Rating(user=51060694, product=54973092, rating=5.0),
  Rating(user=1674962, product=35491224, rating=2.0),
  Rating(user=73533580, product=6166402, rating=4.0),
  Rating(user=75288382, product=40695676, rating=5.0),
  Rating(user=64829209, product=84450372, rating=5.0),
  Rating(user=28118386, product=84902354, rating=5.0),
  Rating(user=82883556, product=23170296, rating=5.0),
  Rating(user=24561755, product=35491224, rating=5.0),
  Rating(user=29680145, product=91031353, rating=5.0),
  Rating(user=51442077, product=37058685, rating=1.0)])

In [14]:
test_data.count()

1581017

## 2. Buiding Recommeder using matrix factorization method

In [18]:
"""

Recommendation model using ALS
    ` rank = 10
    ` number of iteration = 10 

"""

rank = 10
numIterations = 10

model = ALS.train(train_data, rank, numIterations)

#Test model
model.recommendProducts(51442077, 5)

[Rating(user=51442077, product=20582478, rating=4.047894767618655),
 Rating(user=51442077, product=87153075, rating=3.874011811327419),
 Rating(user=51442077, product=75452248, rating=3.684316032208796),
 Rating(user=51442077, product=75582545, rating=3.64617298972512),
 Rating(user=51442077, product=19424917, rating=3.580539208179187)]

## 3. Model Evaluation 

In this part we evaluate the recommendation model on both train dataset and test dataset. 
The RMSE lost function will be used.


In [19]:
from pyspark.mllib.evaluation import RegressionMetrics

In [28]:
# Evaluate the model on training data

predictions = model.predictAll(X_train).map(lambda r: ((r[0], r[1]), r[2]))

labelsAndPreds = train_data.map(lambda (p, u, r): ((p,u),r)) \
                                .join(predictions).map(lambda (k, v): v)


metrics = RegressionMetrics(labelsAndPreds)

rmse = metrics.rootMeanSquaredError

print("Root Mean Squared Error = " + str(rmse))

Root Mean Squared Error = 0.346832703345


In [29]:
# Evaluate the model on test data

test_preds = model.predictAll(X_test).map(lambda r: ((r[0], r[1]), r[2]))

test_labelsAndPreds = test_data.map(lambda (p, u, r): ((p,u),r)) \
                                .join(test_preds).map(lambda (k, v): v) \

test_metrics = RegressionMetrics(test_labelsAndPreds)

test_rmse = test_metrics.rootMeanSquaredError

print("Root Mean Squared Error = " + str(test_rmse))

Root Mean Squared Error = 1.08251940251


In [ ]:
"""
===> Need to try other hyper parameters to find the best model 
"""

In [39]:
"""
Save the model 
    ` Clean the directory before write
    
"""

import os
import sys

MODEL_DIR = os.path.join(ROOT_DIR, 'model', 'als')

!rm -r {MODEL_DIR + "/*"}

try:
    model.save(sc, MODEL_DIR)
    print ("Save model successfully at ", MODEL_DIR)
except:
    sys.exc_info()

Save model successfully
